# MeshcatVisualizer a simple example

In [ ]:
from idyntree.visualize import MeshcatVisualizer
import numpy as np
import os
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from __future__ import print_function
import numpy as np

In [ ]:
def clone_model():
    """Get the model from icub-models. This is usefull to have a standalone example."""
    import zipfile
    import urllib.request
    import tempfile
    model_url = 'https://github.com/robotology/icub-models/archive/refs/tags/v1.20.0.zip'
    model = urllib.request.urlopen(model_url)
    temp = tempfile.NamedTemporaryFile()
    temp_folder = tempfile.TemporaryDirectory()
    temp.write(model.read())
    with zipfile.ZipFile(temp.name, 'r') as zip_obj:
        zip_obj.extractall(temp_folder.name)

    build_path = os.path.join(temp_folder.name, 'icub-models-1.20.0', 'build')
    os.mkdir(build_path)
    os.chdir(build_path)

    os.system('cmake ..')
    return temp_folder, build_path


def expand_enviroment_variable(build_path):
    """Expand the enviroment data. This is usefull to have a standalone example."""
    os.environ["YARP_DATA_DIRS"] += os.path.join(build_path, 'iCub')
    os.environ["ROS_PACKAGE_PATH"] += build_path
    os.environ["AMENT_PREFIX_PATH"] += build_path

def get_model_path(build_path, robot_name='iCubGazeboV2_5'):
    return os.path.join(build_path, 'iCub', 'robots', robot_name, 'model.urdf')

In [ ]:
temp_folder, model_parent_dir = clone_model()
expand_enviroment_variable(model_parent_dir)

In [ ]:
model_path = get_model_path(model_parent_dir)
joint_list = ["torso_pitch", "torso_roll", "torso_yaw",
              "l_shoulder_pitch", "l_shoulder_roll", "l_shoulder_yaw", "l_elbow",
              "r_shoulder_pitch", "r_shoulder_roll", "r_shoulder_yaw", "r_elbow",
              "l_hip_pitch", "l_hip_roll", "l_hip_yaw", "l_knee", "l_ankle_pitch", "l_ankle_roll",
              "r_hip_pitch", "r_hip_roll", "r_hip_yaw", "r_knee", "r_ankle_pitch", "r_ankle_roll"]
joint_dictionary = {joint: (-0.5, 0.5, 0.01) for joint in joint_list}

In [ ]:
viz = MeshcatVisualizer()
viz.set_model_from_file(model_path, joint_list)
viz.load_model(color=[1, 1, 1, 0.8])

In [ ]:
def update_the_model(**joint_values):
    R = np.eye(3)
    p = np.array([0.0, 0.0, 0.0])
    s = np.array(list(joint_values.values()))
    viz.display(p, R, s)

In [ ]:
viz.jupyter_cell()

In [ ]:
interact(update_the_model, **joint_dictionary);